Nhiệm vụ 4 – Merge dữ liệu: 
Mục tiêu : tạo một file dữ liệu hoàn chỉnh phục vụ phân tích sâu hơn. 
- Merge patient_info với visit_records theo patient_id.
- Merge với billing_info theo visit_id.
- Phát hiện các lượt khám thiếu thông tin thanh toán.
- Xuất file hoàn chỉnh.

In [1]:
import pandas as pd
import os

# 1. Thiết lập đường dẫn
base_path = r"D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean"

# 2. Đọc dữ liệu
df_p = pd.read_csv(f"{base_path}\\patient_clean.csv")
df_v = pd.read_csv(f"{base_path}\\visit_clean.csv")
df_b = pd.read_csv(f"{base_path}\\billing_clean.csv")

print(f"Dữ liệu gốc: Patient={len(df_p)}, Visit={len(df_v)}, Billing={len(df_b)}")

#  BƯỚC 1: KIỂM TRA DỮ LIỆU TRƯỚC KHI MERGE (Tránh thừa dữ liệu) 

if df_p['patient_id'].duplicated().any():
    print("CẢNH BÁO: Bảng Patient có ID trùng lặp. Đang tiến hành loại bỏ...")
    df_p = df_p.drop_duplicates(subset=['patient_id'])
if df_b['visit_id'].duplicated().any():
    print("CẢNH BÁO: Bảng Billing có ID trùng lặp. Đang tiến hành loại bỏ...")
    df_b = df_b.drop_duplicates(subset=['visit_id'])

# --- BƯỚC 2: MERGE DỮ LIỆU (Đảm bảo tính 'nhất quán') ---

# Merge 1: Visit + Patient
# validate='many_to_one': Đảm bảo logic nhiều lượt khám thuộc về 1 bệnh nhân duy nhất
df_vp = pd.merge(
    df_v, 
    df_p, 
    on='patient_id', 
    how='left', 
    suffixes=('', '_patient_dup'), # Đánh dấu các cột trùng tên để xử lý sau
    validate='many_to_one'
)

# Merge 2: (Visit + Patient) + Billing
# validate='one_to_one': Giả định mỗi visit chỉ map với 1 bill (nếu 1 visit có nhiều bill, đổi thành 'one_to_many')
df_final = pd.merge(
    df_vp, 
    df_b, 
    on='visit_id', 
    how='left', 
    suffixes=('', '_bill_dup'),
    indicator=True  # Dùng để phát hiện thiếu billing
)

#  BƯỚC 3: XỬ LÝ SAU MERGE (Làm sạch & Kiểm tra thừa thiếu) 

# 1. Loại bỏ các cột dư thừa sinh ra do trùng tên (các cột có đuôi _dup)
cols_to_drop = [c for c in df_final.columns if '_dup' in c]
df_final = df_final.drop(columns=cols_to_drop)

# 2. Kiểm tra tính toàn vẹn (Số dòng sau merge phải bằng số dòng Visit ban đầu)
print("-" * 30)
print("KIỂM TRA CHẤT LƯỢNG MERGE:")
print(f"Số dòng Visit gốc: {len(df_v)}")
print(f"Số dòng sau Merge: {len(df_final)}")

if len(df_v) == len(df_final):
    print(">> ĐÁNH GIÁ: Merge THÀNH CÔNG. Không bị nhân đôi dòng (Không thừa).")
else:
    print(">> CẢNH BÁO: Số lượng dòng bị thay đổi. Cần kiểm tra lại dữ liệu gốc.")

# 3. Phát hiện dữ liệu thiếu hợp lý (Visit nhưng không có Patient)
missing_patient = df_final[df_final['patient_id'].isnull()]
if not missing_patient.empty:
    print(f">> CẢNH BÁO: Có {len(missing_patient)} lượt khám không tìm thấy thông tin bệnh nhân (Patient ID sai/thiếu).")

# 4. Phát hiện thiếu thông tin thanh toán (Yêu cầu đề bài)
# Logic: Có Visit nhưng Billing là NaN (check qua cột _merge = left_only)
missing_billing = df_final[df_final['_merge'] == 'left_only']
print(f">> KẾT QUẢ: Phát hiện {len(missing_billing)} lượt khám chưa có thông tin thanh toán.")

# Xóa cột chỉ báo _merge trước khi xuất
df_final = df_final.drop(columns=['_merge'])

# --- BƯỚC 4: XUẤT FILE ---
output_path = f"{base_path}\\merge_task04.csv"
df_final.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"Đã xuất file hoàn chỉnh: {output_path}")

Dữ liệu gốc: Patient=40, Visit=120, Billing=120
CẢNH BÁO: Bảng Billing có ID trùng lặp. Đang tiến hành loại bỏ...
------------------------------
KIỂM TRA CHẤT LƯỢNG MERGE:
Số dòng Visit gốc: 120
Số dòng sau Merge: 120
>> ĐÁNH GIÁ: Merge THÀNH CÔNG. Không bị nhân đôi dòng (Không thừa).
>> KẾT QUẢ: Phát hiện 44 lượt khám chưa có thông tin thanh toán.
Đã xuất file hoàn chỉnh: D:\OneDrive\Máy tính\18A2_Nhom_4_Case_9\data_clean\merge_task04.csv
